# BERT모델 임베딩

In [ ]:
# 1. Google Drive 마운트 (최초 실행 시 인증 필요)
from google.colab import drive
drive.mount('/content/drive')

# 2. 필요한 라이브러리 임포트
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
import torch.optim as optim  # torch의 AdamW 사용
from torch.cuda.amp import autocast, GradScaler

# 3. 커스텀 데이터셋 클래스 정의
class VideoDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_length=128):
        self.data = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # title, tags, img_text를 하나의 문자열로 결합
        text = f"{row['title']} {row['tags']} {row['img_text']}"
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        # extra 차원 제거
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        # 'category' 컬럼 값이 1~10 범위라고 가정 (0부터 시작하도록 조정)
        label = torch.tensor(row['category'] - 1)
        return inputs, label

# 4. BERT 기반 분류 모델 정의
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 10)  # 10개 카테고리

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰 임베딩
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 5. 경로 설정 및 데이터셋 준비 (경로는 자신의 Google Drive 내 파일 위치에 맞게 수정)
csv_path = "/content/drive/MyDrive/path_to_your_csv/filtered_video_data.csv"
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
dataset = VideoDataset(csv_path, tokenizer, max_length=128)

# DataLoader 최적화: 배치 사이즈 32, num_workers=4, pin_memory 활성화
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

# 6. 모델, 옵티마이저, 손실 함수 및 mixed precision을 위한 GradScaler 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertClassifier().to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
scaler = GradScaler()  # mixed precision을 위한 스케일러

# 7. 학습 루프 (Mixed Precision Training 적용)
epochs = 3
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        inputs, labels = batch
        input_ids = inputs["input_ids"].to(device, non_blocking=True)
        attention_mask = inputs["attention_mask"].to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        # 자동 혼합 정밀도 적용
        with autocast():
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# 8. 학습 완료 후 모델 저장 (Google Drive 경로에 저장)
torch.save(model.state_dict(), "/content/drive/MyDrive/path_to_your_csv/bert_classifier.pth")